# Evaluate the data

In [ ]:
import os
os.environ["MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR"] = "False"
import mlflow 
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sys
from helpers import get_transferability_data_from_mlflow, get_threshold_val, get_attacker_regret_errors
import pandas as pd
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
GLOBAL_ATTACKER_ORDER = ["clean", "random", "fgsm_pred_ls", "fgsm_decision", "fgsm_custom", "apgd_pred_l2", "apgd_decision", "apgd_custom"]
model_name_dict = {
    'baseline_mse' : "PF",
    'SPO' : "SPO",
    'DBB' : "DBB",
    "IMLE" : "IMLE",
    'FenchelYoung' : "FY",
    "DCOL" : "QPTL",
    'IntOpt' : "IntOpt",
    'CachingPO_listwise' : "Listwise",
    'CachingPO_pairwise' : "Pairwise",
    'CachingPO_pairwise_diff' : "PairwiseDiff",
    'CachingPO_MAP_c' : "MAP",
}

In [ ]:
# Also create a column for the attacker_sig
# Helper function to safely get parameter values
def safe_get_param(row, param_name, default="None"):
    if param_name in row and pd.notna(row[param_name]):
        return str(row[param_name])
    return default

def format_str_to_same_length(inp1, inp2, inp3, padding1, padding2, padding3):
    len_1 = len(inp1)
    len_2 = len(inp2)
    len_3 = len(inp3)
    # pad all of them using _
    inp1 = inp1 + "_" * (padding1 - len_1)
    inp2 = inp2 + "_" * (padding2 - len_2)
    inp3 = inp3 + "_" * (padding3 - len_3)
    # return one string
    return f"{inp1}_{inp2}_{inp3}"

def insert_spacers(z, labels, group_keys):
    new_z = []
    new_labels = []
    prev_key = None

    group_id = 0
    for i, (row, label, key) in enumerate(zip(z, labels, group_keys)):
        if prev_key is not None and key != prev_key:
            new_z.append([np.nan] * z.shape[1])  # insert blank row
            new_labels.append(f"Group: {group_id+1}")                # blank label
            group_id += 1
        new_z.append(row)
        new_labels.append(label)
        prev_key = key

    return np.array(new_z), new_labels

def load_data(dataset_name):
    dataset_name = dataset_name + "_Models"
    # TODO: Adjust to your experiment id
    experiment_id = "228319214249946667"
    # Get all the models with img_size 12 -> get a list of run ids
    runs = mlflow.search_runs(experiment_ids=experiment_id)
    # Filter for finished
    runs = runs[runs["status"] == "FINISHED"]
    # Filter for dataset_name
    runs = runs[runs["params.attacked_models_experiment"] == dataset_name]
    # Assert that all models have been attacked
    runs["attacker_sig"] = runs.apply(lambda row: 
        row["params.attacker"] + 
        "_e" + safe_get_param(row, "params.epsilon") +
        "_a" + safe_get_param(row, "params.alpha") +
        "_m" + safe_get_param(row, "params.max_iter") +
        "_r" + safe_get_param(row, "params.restarts") +
        ("_s" + safe_get_param(row, "params.use_signed_grad") if "params.use_signed_grad" in row else ""), 
        axis=1)
    # In case of warcraft change the attacked_models_name
    if dataset_name == "Warcraft_Models":
        runs["params.attacked_models_name"] = runs["params.attacked_models_name"].apply(lambda x: x.replace("_regret", ""))
    runs["model_epsilon"] = runs["params.attacked_models_name"].astype(str) + "_eps=" + runs["params.epsilon"].astype(str)
    print(f"Found {len(runs)} runs for {dataset_name}")
    return runs

def validate_attacker_for_epsilon(data):
    # In this case we do not care about the random noise attacker
    data = data[data["params.attacker"] != "random_noise"]
    # for each attacker check that rre and frre increase with each epsilon increases
    attackers = data["params.attacker"].unique()
    for attacker in attackers:
        print(f"Evaluating attacker: {attacker}")
        attacker_data = data[data["params.attacker"] == attacker]
        # now check for each model
        models = attacker_data["params.attacked_models_name"].unique()
        for model in models:
            # Now filter 
            model_data = attacker_data[attacker_data["params.attacked_models_name"] == model]
            # Sort based on epsilon
            model_data = model_data.sort_values("params.epsilon")
            frre = model_data["metrics.mean_fool_rel_regret"].values
            rre = model_data["metrics.mean_rel_regret"].values
            acc = model_data["metrics.mean_acc_error"].values
            facc = model_data["metrics.mean_fool_error"].values
            epsilons = model_data["params.epsilon"].values
            # Check if frre and rre are increasing
            for i in range(1, len(frre)):
                if frre[i] < frre[i-1]:
                    print(f"Frre is not increasing for {attacker} on {model} for epsilon {model_data['params.epsilon'].values[i]}")
                    print(f"Frre: (eps: {epsilons[i]}) {frre[i]} < {frre[i-1]} (eps: {epsilons[i-1]})")
                    print("COMPARING ACCURACY")
                    print(f"Acc: (eps: {epsilons[i]}) {acc[i]} : {acc[i-1]} (eps: {epsilons[i-1]})")
                if rre[i] < rre[i-1]:
                    print(f"Rre is not increasing for {attacker} on {model} for epsilon {model_data['params.epsilon'].values[i]}")
                    print(f"Rre: (eps: {epsilons[i]}) {rre[i]} < {rre[i-1]} (eps: {epsilons[i-1]})")
                    print("COMPARING ACCURACY")
                    print(f"Acc: (eps: {epsilons[i]}) {acc[i]} : {acc[i-1]} (eps: {epsilons[i-1]})")
                    print(f"Facc: (eps: {epsilons[i]}) {facc[i]} : {facc[i-1]} (eps: {epsilons[i-1]})")
                    print("-" * 20)
        print("*" * 20)

In [ ]:
# DEFINE THE DATASET HERE

DATASET = "Warcraft"

## General Model Performances

In [ ]:
data = load_data(DATASET)

rel_metrics = ["test_regret", "test_mse"]
attacked_models_run_ids = data["params.attacked_models_run_id"].unique()

modelnames = []
metric_vals  = []

for metric in rel_metrics:
    # Create one plot for each 
    for run_id in attacked_models_run_ids: 
        # get the data 
        model_run = mlflow.get_run(run_id) 
        model_name = model_run.data.params["modelname"]
        loss = model_run.data.params.get("loss", None)
        if loss is not None:
            unique_name = f"{model_name}_{loss}"
        else:
            unique_name = model_name
        modelnames.append(unique_name)
        # Get the metric values 
        metric_vals.append(model_run.data.metrics[metric])
    # Create the plot
    fig = px.bar(
        x=modelnames,
        y=metric_vals,
        title=f"Metric: {metric}",
        labels={"x": "Model", "y": metric},
    )
    fig.show()

## Mean

This plot will show the mean performance for the given metric and epsilon for each model and for the different attackers.

In [ ]:
# DEFINE THIS 
metric = "metrics.mean_rel_regret" # metrics.mean_fool_rel_regret
data = load_data(DATASET)

grouped = data.groupby("model_epsilon")
for model_eps, group in grouped:
    assert group["params.attacked_models_run_id"].nunique() == 1, f"Expected only one run id for {model_eps}, but got {group['params.attacked_models_run_id'].nunique()}"
    # Get the values
    run_id_one_attacker = group["run_id"].values[0]
    # Get the
    rre_clean, _, _,_, _  =  get_attacker_regret_errors(run_id_one_attacker, problem=DATASET)
    # Get the mean 
    mean_rre_attacked_model = np.mean(rre_clean)
    # Now append a row to the dataframe 
    for epsilon in group["params.epsilon"].unique():
        params = {
            "params.epsilon": epsilon,
            "params.attacker": "Clean Model",
            "params.attacked_models_name": group["params.attacked_models_name"].values[0],
            "metrics.mean_rel_regret": mean_rre_attacked_model
        }
        # Append the row to the dataframe
        data = pd.concat([data, pd.DataFrame([params])], ignore_index=True)

existing_models = [model for model in model_name_dict if model in data["params.attacked_models_name"].unique()]
# Create model positions based on your desired order
model_positions = {model: i for i, model in enumerate(existing_models)}

# Create aliases for display
model_aliases = [model_name_dict[model] for model in existing_models]


# Create attacker signature with only parameters that exist
data["attacker_sig"] = data.apply(lambda row: 
    row["params.attacker"] + 
    "_e" + safe_get_param(row, "params.epsilon") +
    "_a" + safe_get_param(row, "params.alpha") +
    "_m" + safe_get_param(row, "params.max_iter") +
    "_r" + safe_get_param(row, "params.restarts") +
    ("_s" + safe_get_param(row, "params.use_signed_grad") if "params.use_signed_grad" in row else "") + 
    # also add the params.initial_window_size if it exists
    ("_i" + safe_get_param(row, "params.initial_window_size") if "params.initial_window_size" in row else ""),
    axis=1)

# Budget function
def get_budget(row):
    # check which attacker is used 
    if row["params.attacker"] == "Clean Model":
        return "clean"
    elif ("fgsm" in row["params.attacker"]) or ("random" in row["params.attacker"]) or ("Iterative" in row["params.attacker"]):
        return "medium"
    elif ("apgd" in row["params.attacker"]):
        if row["params.max_iter"] == "200":
            return "high"
        elif row["params.max_iter"] == "100":
            return "medium"
        elif row["params.max_iter"] == "10":
            return "low"
    else:
        return "medium"

# Add the budget column to the dataframe
data["budget"] = data.apply(get_budget, axis=1)

# Sort data by epsilon to ensure proper ordering in facets
data["params.epsilon"] = pd.to_numeric(data["params.epsilon"], errors='coerce')
data = data.sort_values("params.epsilon")

# Define offsets for each budget level
budget_offsets = {"low": -0.2, "medium": 0.0, "high": 0.2, "clean": 0.0}


# Create the actual x-positions with offsets
data["x_position"] = data.apply(lambda row: 
    model_positions[row["params.attacked_models_name"]] + budget_offsets[row["budget"]], 
    axis=1)

# Define symbols for each budget level (including clean)
budget_symbols = {"low": "triangle-left", "medium": "circle", "high": "triangle-right", "clean": "x"}
data["symbol"] = data["budget"].map(budget_symbols)

# Create a scatter plot
fig = px.scatter(
    data,
    x="x_position",  # Use jittered x-positions
    y=metric,
    facet_col="params.epsilon",
    color="params.attacker",
    symbol="budget",
    hover_name="attacker_sig",
    symbol_map=budget_symbols,
    size_max=120, 
)

# Update traces to make Clean Model black
for trace in fig.data:
    if 'Clean Model' in trace.name:
        trace.marker.color = 'black'

# Update x-axis to show model names instead of numeric positions

fig.update_xaxes(
    tickmode='array',
    tickvals=list(model_positions.values()),
    ticktext=model_aliases,  # Use aliases here
    title="Models"
)

# Update the layout
fig.update_layout(
    height=1300,
)

In [ ]:
# DEFINE
epsilon = "8"
data = load_data(DATASET)

# Filter the data once
data = data[data["params.epsilon"] == epsilon]
data = data[(data["params.max_iter"] == "100") | (data["params.max_iter"].isna()) | (data["params.max_iter"] == "50")]
data = data[data["params.attacker"].str.contains("apgd") | data["params.attacker"].str.contains("argeted")]

# Define the model name mapping
model_name_dict = {
    'baseline_mse': "PF",
    'SPO': "SPO",
    'DBB': "DBB",
    "IMLE": "IMLE",
    'FenchelYoung': "FY",
    "DCOL": "DCOL",
    'IntOpt': "IntOpt",
    'CachingPO_listwise': "Listwise",
    'CachingPO_pairwise': "Pairwise",
    'CachingPO_pairwise_diff': "PairwiseDiff",
    'CachingPO_MAP_c': "MAP",
}

# Create a readable model name column for the x-axis
data["model_name"] = data["params.attacked_models_name"].map(model_name_dict).fillna(data["params.attacked_models_name"]).astype(str)

# Prepare a list of records for a new DataFrame that will hold
# every distribution point for each (attacker, run_id, metric) *plus* clean distribution
rows_for_boxplot = []

# Process both metrics
for metric in ["rel_regret", "fool_rel_regret"]:
    # Add clean data only once for rel_regret (since it's the same for both)
    if metric == "rel_regret":
        grouped = data.groupby("model_name")
        for name, group_data in grouped:
            # get the first entry -> just need the clean val 
            run_id_one_attacker = group_data["run_id"].values[0]
            # Get the values
            rre_clean, _, _, _, _ = get_attacker_regret_errors(run_id_one_attacker, problem=DATASET)
            for val in rre_clean:
                rows_for_boxplot.append({
                    "model_name": name,
                    "attacker": "clean",
                    "regret_value": val,
                    "metric": metric  # Add metric column
                })
    
    # Process attacked data for each metric
    for idx, row in data.iterrows():
        run_id = row["run_id"]
        attacker = row["attacker_sig"]
        model_name = row["model_name"]
        
        # Get the regret errors
        _, _, rre_attacked, _, frre_attacked = get_attacker_regret_errors(run_id, problem=DATASET)
        
        # Collect the attacked distribution based on metric
        if metric == "fool_rel_regret":
            for val in frre_attacked:
                rows_for_boxplot.append({
                    "model_name": model_name,
                    "attacker": attacker,
                    "regret_value": val,
                    "metric": metric  # Add metric column
                })
        elif metric == "rel_regret":
            for val in rre_attacked:
                rows_for_boxplot.append({
                    "model_name": model_name,
                    "attacker": attacker,
                    "regret_value": val,
                    "metric": metric  # Add metric column
                })

# Convert the collected rows into a DataFrame
df_box = pd.DataFrame(rows_for_boxplot)

# Set the order of model names to match the dictionary order
# Filter to only include models that are actually present in the data
model_order = [model for model in model_name_dict.values() if model in df_box["model_name"].unique()]
df_box["model_name"] = pd.Categorical(df_box["model_name"], categories=model_order, ordered=True)

print(df_box.columns)

# Set up color scheme
colorscheme = px.colors.qualitative.Plotly
color_map = {attacker: colorscheme[i] for i, attacker in enumerate(GLOBAL_ATTACKER_ORDER)}
color_map["clean"] = "black"

# Create the faceted boxplot
fig = px.box(
    df_box,
    x="model_name",
    y="regret_value",
    color="attacker",
    facet_col="metric",  # This creates separate subplots for each metric
    title="Boxplots of RRE distributions (clean vs. attacked) - Both Metrics",
    color_discrete_map=color_map,
)

# Update layout
fig.update_layout(
    legend_title="Attacker",
    xaxis_title="Attacked Model",
    template="plotly_white",
    height=1000,
)

# Ensure the x-axis follows the model order
fig.update_xaxes(categoryorder='array', categoryarray=model_order)

# Optional: Update facet titles to be more readable
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

fig.show()

In [ ]:
# DEFINE THIS
epsilon = "8"
startpoint = -0.1
endpoint = 4

# Do not change
metric = "fool_rel_regret"
data = load_data(DATASET)
# Now filter the data for the given attackers 
data = data[data["params.epsilon"] == epsilon]
data = data[(data["params.max_iter"] == "100") | (data["params.max_iter"].isna()) | (data["params.max_iter"] == "50")]
# filte only for the apgd attacker and targeted
#data = data[data["params.attacker"].str.contains("apgd") | data["params.attacker"].str.contains("argeted")]
runs = data.copy()


# Now create the CCDF plot
# Now load all the runs in this experiment
runs["attacker_sig"] = runs.apply(lambda row: 
    row["params.attacker"] + 
    "_e" + safe_get_param(row, "params.epsilon") +
    "_a" + safe_get_param(row, "params.alpha") +
    "_m" + safe_get_param(row, "params.max_iter") +
    "_r" + safe_get_param(row, "params.restarts") +
    ("_s" + safe_get_param(row, "params.use_signed_grad") if "params.use_signed_grad" in row else ""), 
    axis=1)

# Get all unique attacker signatures for consistent colors across subplots
all_attacker_sigs = runs[runs["params.epsilon"] == epsilon]["attacker_sig"].unique()
color_dict = {attacker: px.colors.qualitative.Plotly[i % len(px.colors.qualitative.Plotly)] 
              for i, attacker in enumerate(all_attacker_sigs)}

# Create a figure with subplots for each epsilon value
# Get the number of unique models that are attacked
unique_models = runs["params.attacked_models_name"].unique()
rows = (len(unique_models) + 3) // 4  # Calculate needed rows (ceiling division)
fig = make_subplots(rows=rows, cols=4, subplot_titles=list(unique_models))

# Filter data for this epsilon
for i, attacked_modelname in enumerate(unique_models):
    plot_data = runs[runs["params.epsilon"] == epsilon].copy()
    plot_data = plot_data[plot_data["params.attacked_models_name"] == attacked_modelname]
    
    # Get the row and column to add the trace to
    row_num = i // 4 + 1
    col_num = i % 4 + 1
    
    # For each attacker create a CCDF line and add it to the subplot
    for _, row in plot_data.iterrows():
        rre_clean, _, rre_adv, _, frre_adv = get_attacker_regret_errors(row["run_id"], problem=DATASET)
        # Compute the CCDF (1 - empirical CDF)
        x = np.linspace(startpoint, endpoint, 1000)
        y = []
        for x_val in x:
            # calculate the fraction of samples greater than x_val 
            if metric == "fool_rel_regret":
                threshold_val = get_threshold_val(frre_adv, x_val)
            elif metric == "rel_regret":
                threshold_val = get_threshold_val(rre_adv, x_val)
            y.append(threshold_val)
        
        # Create the CCDF line plot with consistent colors
        fig.add_trace(
            go.Scatter(
                x=x,
                y=y,
                mode='lines',
                name=row['attacker_sig'],
                line=dict(color=color_dict[row['attacker_sig']]),
                legendgroup=row['attacker_sig'],  # Group by attacker_sig for legend
                showlegend=(i == 1),  # Only show in legend for first subplot
            ),
            row=row_num, col=col_num,
        )

# Update layout
fig.update_layout(
    title_text=f"CCDF of {metric} for epsilon {epsilon} across all models",
    xaxis_title=f"Threshold value",
    yaxis_title="Fraction of adversarial inputs >= threshold",
    legend_title="Attacker",
    template="plotly_white",
    legend=dict(
        groupclick="togglegroup"  # Enable clicking on a legend group to toggle all traces
    ),
    # Adjust the height and width of the figure
    height=rows*300,
    width=1800,
)

# Update all subplot axes for consistency
for i in range(1, rows*4+1):
    row = (i-1)//4 + 1
    col = (i-1)%4 + 1
    if i <= len(unique_models):
        fig.update_xaxes(title_text="Threshold", row=row, col=col)
        fig.update_yaxes(title_text="Fraction ≥ threshold", row=row, col=col)

fig.update_layout(
    height=rows*400,
    width=2200,
)
fig.show()

# Adjusted Plots for Thesis

## In thesis 

In [ ]:
# DEFINE THIS
epsilon = "8"
startpoint = -0.05
endpoint = 3.0

# Do not change
metric = "fool_rel_regret"
data = load_data(DATASET)
# Now filter the data for the given attackers 
data = data[data["params.epsilon"] == epsilon]
data = data[(data["params.max_iter"] == "100") | (data["params.max_iter"].isna()) | (data["params.max_iter"] == "50")]
# filte only for the apgd attacker and targeted
data = data[data["params.attacker"].str.contains("apgd") | data["params.attacker"].str.contains("argeted") | (data["params.attacker"].str.contains("andom"))]
runs = data.copy()


# Now create the CCDF plot
# Now load all the runs in this experiment
runs["attacker_sig"] = runs.apply(lambda row: 
    row["params.attacker"] + 
    "_e" + safe_get_param(row, "params.epsilon") +
    "_a" + safe_get_param(row, "params.alpha") +
    "_m" + safe_get_param(row, "params.max_iter") +
    "_r" + safe_get_param(row, "params.restarts") +
    ("_s" + safe_get_param(row, "params.use_signed_grad") if "params.use_signed_grad" in row else ""), 
    axis=1)

In [ ]:
def attacker_sig_to_alias(sig):
    if "terative" in sig:
        return "TARGETED"
    elif "apgd_dec" in sig:
        return "APGD-TRAIN"
    elif "apgd_adv_loss_mean" in sig:
        return "APGD-L2"
    elif "apgd_adv_loss_enf" in sig:
        return "APGD-NOTOPT"
    elif "random" in sig:
        return "RANDOM"
    else:
        raise ValueError(f"Unknown attacker sig: {sig}")

In [ ]:
# Create a figure with subplots for each epsilon value
desired_order = list(model_name_dict.keys())
unique_models_in_data = runs["params.attacked_models_name"].unique()
# Keep only models that exist in data, in the desired order
unique_models = [model for model in desired_order if model in unique_models_in_data]

all_attacker_sigs = runs[runs["params.epsilon"] == epsilon]["attacker_sig"].unique()
all_attacker_sigs = sorted(all_attacker_sigs, key=lambda sig: attacker_sig_to_alias(sig))
color_dict = {attacker: px.colors.qualitative.Plotly[i % len(px.colors.qualitative.Plotly)] 
              for i, attacker in enumerate(all_attacker_sigs)}
rows = (len(unique_models) + 2) // 2  # Calculate needed rows (ceiling division)

fig = make_subplots(
    rows=rows, cols=2, 
    subplot_titles=list(unique_models),
    shared_xaxes=True,
    shared_yaxes=True,
    #x_title="Threshold value",  # Shared x-axis title
    #y_title="Fraction of adversarial inputs ≥ threshold",  # Shared y-axis title
    horizontal_spacing=0.025,
    vertical_spacing=0.025
)

# Filter data for this epsilon
for i, attacked_modelname in enumerate(unique_models):
    plot_data = runs[runs["params.epsilon"] == epsilon].copy()
    plot_data = plot_data[plot_data["params.attacked_models_name"] == attacked_modelname]
    # sort the plot data by attacker alphabetically
    plot_data = plot_data.sort_values("attacker_sig")

    # Get the row and column to add the trace to
    row_num = i // 2 + 1
    col_num = i % 2 + 1
    
    # For each attacker create a CCDF line and add it to the subplot
    for _, row in plot_data.iterrows():
        rre_clean, _, rre_adv, _, frre_adv = get_attacker_regret_errors(row["run_id"], problem=DATASET)
        x = np.linspace(startpoint, endpoint, 1000)
        y = []
        for x_val in x:
            # calculate the fraction of samples greater than x_val 
            if metric == "fool_rel_regret":
                threshold_val = get_threshold_val(frre_adv, x_val)
            elif metric == "rel_regret":
                threshold_val = get_threshold_val(rre_adv, x_val)
            y.append(threshold_val)
        
        # Create the CCDF line plot with consistent colors
        fig.add_trace(
            go.Scatter(
                x=x,
                y=y,
                mode='lines',
                name=attacker_sig_to_alias(row['attacker_sig']),
                line=dict(color=color_dict[row['attacker_sig']]),
                legendgroup=row['attacker_sig'],  # Group by attacker_sig for legend
                showlegend=(i == 1),  # Only show in legend for first subplot
            ),
            row=row_num, col=col_num,
        )

# Then just hide labels except on edges
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False) 
fig.update_xaxes(showticklabels=True, row=rows)  # Bottom row only
fig.update_yaxes(showticklabels=True, col=1)    # Left column only

fig.update_layout(
    height=900,
    width=600,
    showlegend=False,
    template="plotly_white",
    #margin=dict(l=50, r=20, t=40, b=50)
)

# Update the titles 
for i, annotation in enumerate(fig.layout.annotations):
    if annotation.text in model_name_dict:
        fig.layout.annotations[i].text = model_name_dict[annotation.text]
fig.update_annotations(font=dict(size=10))


fig.update_layout(
    showlegend=True,
    legend=dict(
        x=0.60,  # Right edge (0-1 scale)
        y=-0.01,  # Bottom edge (0-1 scale)
        xanchor='left',
        yanchor='bottom',
        bgcolor='rgba(255, 255, 255, 0.8)',  # Semi-transparent white background
        bordercolor='rgba(0, 0, 0, 0.2)',
        borderwidth=1,
        font=dict(size=9),  # Smaller font to save space
        tracegroupgap=5,  # Reduce space between items (default is ~10-15)

    ),
    # Reduce margins significantly
    margin=dict(
        l=50,   # Left margin
        r=20,   # Right margin (reduced since legend is inside plot area)
        t=30,   # Top margin
        b=50    # Bottom margin (for x-axis label)
    ),
    # Optional: make figure more compact
    height=900,  # Reduce if needed
    width=600
)

fig.show()

In [ ]:
# TODO: Define path
path = ""
fig.write_image(path, format="pdf", width=600, height=900)

# Tables for Thesis

In [ ]:
# TABLES for the THESIS
# DEFINE THIS 
metric = "metrics.mean_rel_regret" # metrics.mean_fool_rel_regret
data = load_data(DATASET)

In [ ]:
import tempfile
from adv_error_metrics import adv_relative_regret_error
from helpers import get_min_or_max_problem


eps = "8"
attacker_sig = "random_noise_e8_aNone_mNone_r10_sNone"
show_percentiles = False
text_bf = False
problem = DATASET


# filter for one epsilon
data_table = data[data["params.epsilon"] == eps]
if attacker_sig != "Clean_Model":
    data_table = data_table[data_table["attacker_sig"] == attacker_sig]
if attacker_sig == "Clean_Model":
    # In this case it does not matter which attacker we take
    # Just group and take the first group 
    any_to_filter = data_table["attacker_sig"].unique()[0]
    data_table = data_table[data_table["attacker_sig"] == any_to_filter]

# Now make sure the entries are ordered based on the models in model_name_dict
data_table["model_order"] = data_table["params.attacked_models_name"].apply(lambda x
    : list(model_name_dict.keys()).index(x) if x in model_name_dict else np.nan)
data_table = data_table.sort_values("model_order")

attacker_names = ""
mean_rre_string = ""
lower_percentile_rre_string = "{\scriptsize \quad 25th \%}"
upper_percentile_rre_string = "{\scriptsize \quad 75th \%}"
if attacker_sig == "Clean_Model":
    # For each attacker 
    for idx, row in data_table.iterrows():
        attacker_run_id = row["run_id"]
        # Download the attacker_data_for_the_run

        minimize = get_min_or_max_problem(problem)
        # First download the data
        with tempfile.TemporaryDirectory() as tmpdir:
            path = mlflow.artifacts.download_artifacts(
                run_id=attacker_run_id,
                artifact_path="adv_samples/adv_samples.npz",
                dst_path=tmpdir,
            )
            adv_samples = np.load(open(path, "rb"))
            path = mlflow.artifacts.download_artifacts(
                run_id=attacker_run_id,
                artifact_path="adv_samples/adv_decisions.npz",
                dst_path=tmpdir,
            )
            adv_decisions = np.load(open(path, "rb"))
            # Now compute the relative regrets and regrets
            c = adv_samples["c"]
            dec = adv_samples["dec"]
            dec_hat = adv_decisions["dec_hat"]
            dec_adv_hat = adv_decisions["dec_adv_hat"]
            rel_regrets = adv_relative_regret_error(
                c=c,
                dec_adv=dec,
                dec_adv_hat=dec_hat,  # we only want the normal regret and not of the adv
                minimize=minimize,
            )

            # Now compute the 25th and 75th percentile
            mean_rre = np.mean(rel_regrets)
            lower_percentile_rre = np.percentile(rel_regrets, 25)
            upper_percentile_rre = np.percentile(rel_regrets, 75)
            # Append to the strings
            attacker_names += f" & {row['params.attacked_models_name']}"
            if text_bf:
                mean_rre_string += f" & \\textbf{{ {mean_rre:.3f} }}"
            else:
                mean_rre_string += f" & {mean_rre:.3f}"
            formatted_lower = f"{lower_percentile_rre:.3f}"
            if formatted_lower == "-0.000":
                formatted_lower = "0.000"
            lower_percentile_rre_string += f" &  \\scriptsize{{ {formatted_lower} }}"
            upper_percentile_rre_string += f" &  \\scriptsize{{ {upper_percentile_rre:.3f} }}"
else:
    # For each attacker 
    for idx, row in data_table.iterrows():
        attacker_run_id = row["run_id"]
        # Download the attacker_data_for_the_run
        with tempfile.TemporaryDirectory() as tmpdir:
            path = mlflow.artifacts.download_artifacts(
                run_id=attacker_run_id,
                artifact_path="error_metrics/error_metrics.npz",
                dst_path=tmpdir,
            )
            adv_samples = np.load(open(path, "rb"))
            rel_regrets = adv_samples["rel_regrets"]
            # Now compute the 25th and 75th percentile
            mean_rre = np.mean(rel_regrets)
            lower_percentile_rre = np.percentile(rel_regrets, 25)
            upper_percentile_rre = np.percentile(rel_regrets, 75)
            # Append to the strings
            attacker_names += f" & {row['params.attacked_models_name']}"
            if text_bf:
                mean_rre_string += f" & \\textbf{{ {mean_rre:.3f} }}"
            else:
                mean_rre_string += f" & {mean_rre:.3f}"
            formatted_lower = f"{lower_percentile_rre:.3f}"
            if formatted_lower == "-0.000":
                formatted_lower = "0.000"
            lower_percentile_rre_string += f" &  \\scriptsize{{ {formatted_lower} }}"
            upper_percentile_rre_string += f" &  \\scriptsize{{ {upper_percentile_rre:.3f} }}"
# Print the results
print(attacker_sig + "\\\\")
print()
print(attacker_names + "\\\\")
print()
print(mean_rre_string + "\\\\" )
if show_percentiles:
    print(lower_percentile_rre_string + "\\\\" )
    print(upper_percentile_rre_string + "\\\\")



In [ ]:
to_plot = ["IMLE" ,"DBB"]
# DEFINE THIS
epsilon = "8"
startpoint = -0.05
endpoint = 4.0

# Do not change
metric = "fool_rel_regret"
data = load_data(DATASET)
# Now filter the data for the given attackers 
data = data[data["params.epsilon"] == epsilon]
data = data[(data["params.max_iter"] == "100") | (data["params.max_iter"].isna()) | (data["params.max_iter"] == "50")]
# filte only for the apgd attacker and targeted
data = data[data["params.attacker"].str.contains("apgd") | data["params.attacker"].str.contains("argeted") | (data["params.attacker"].str.contains("andom"))]
runs = data.copy()


# Now create the CCDF plot
# Now load all the runs in this experiment
runs["attacker_sig"] = runs.apply(lambda row: 
    row["params.attacker"] + 
    "_e" + safe_get_param(row, "params.epsilon") +
    "_a" + safe_get_param(row, "params.alpha") +
    "_m" + safe_get_param(row, "params.max_iter") +
    "_r" + safe_get_param(row, "params.restarts") +
    ("_s" + safe_get_param(row, "params.use_signed_grad") if "params.use_signed_grad" in row else ""), 
    axis=1)
# Create a figure with subplots for each epsilon value
desired_order = list(model_name_dict.keys())
unique_models_in_data = runs["params.attacked_models_name"].unique()
# Keep only models that exist in data, in the desired order
unique_models = [model for model in desired_order if model in unique_models_in_data]
# Now filter the unique models based on to_plot
unique_models = [model for model in unique_models if model_name_dict.get(model, model) in to_plot]



all_attacker_sigs = runs[runs["params.epsilon"] == epsilon]["attacker_sig"].unique()
all_attacker_sigs = sorted(all_attacker_sigs, key=lambda sig: attacker_sig_to_alias(sig))
color_dict = {attacker: px.colors.qualitative.Plotly[i % len(px.colors.qualitative.Plotly)] 
              for i, attacker in enumerate(all_attacker_sigs)}
rows =  1  # Calculate needed rows (ceiling division)
#fig = make_subplots(rows=rows, cols=2, subplot_titles=list(unique_models))
fig = make_subplots(
    rows=rows, cols=2, 
    subplot_titles=list(unique_models),
    shared_xaxes=True,
    shared_yaxes=True,
    #x_title="Threshold value",  # Shared x-axis title
    #y_title="Fraction of adversarial inputs ≥ threshold",  # Shared y-axis title
    horizontal_spacing=0.025,
    vertical_spacing=0.05
)


# Filter data for this epsilon
for i, attacked_modelname in enumerate(unique_models):
    plot_data = runs[runs["params.epsilon"] == epsilon].copy()
    plot_data = plot_data[plot_data["params.attacked_models_name"] == attacked_modelname]
    # sort the plot data by attacker alphabetically
    plot_data = plot_data.sort_values("attacker_sig")

    # Get the row and column to add the trace to
    row_num = i // 2 + 1
    col_num = i % 2 + 1
    
    # For each attacker create a CCDF line and add it to the subplot
    for _, row in plot_data.iterrows():
        rre_clean, _, rre_adv, _, frre_adv = get_attacker_regret_errors(row["run_id"], problem=DATASET)
        x = np.linspace(startpoint, endpoint, 1000)
        y = []
        for x_val in x:
            # calculate the fraction of samples greater than x_val 
            if metric == "fool_rel_regret":
                threshold_val = get_threshold_val(frre_adv, x_val)
            elif metric == "rel_regret":
                threshold_val = get_threshold_val(rre_adv, x_val)
            y.append(threshold_val)
        
        # Create the CCDF line plot with consistent colors
        fig.add_trace(
            go.Scatter(
                x=x,
                y=y,
                mode='lines',
                name=attacker_sig_to_alias(row['attacker_sig']),
                line=dict(color=color_dict[row['attacker_sig']]),
                legendgroup=row['attacker_sig'],  # Group by attacker_sig for legend
                showlegend=(i == 0),  # Only show in legend for first subplot
            ),
            row=row_num, col=col_num,
        )

# Show tick labels on both plots since there's only one row
fig.update_xaxes(showticklabels=True)  # Show x-axis labels on both plots
fig.update_yaxes(showticklabels=True, col=1)    # Left column only for y-axis

fig.update_layout(
    height=900,
    width=600,
    showlegend=False,
    template="plotly_white",
    #margin=dict(l=50, r=20, t=40, b=50)
)

# Update the titles 
for i, annotation in enumerate(fig.layout.annotations):
    if annotation.text in model_name_dict:
        fig.layout.annotations[i].text = model_name_dict[annotation.text]
fig.update_annotations(font=dict(size=10))


# fig.update_layout(
#     showlegend=True,
#     legend=dict(
#         x=0.60,  # Right edge (0-1 scale)
#         y=-0.01,  # Bottom edge (0-1 scale)
#         xanchor='left',
#         yanchor='bottom',
#         bgcolor='rgba(255, 255, 255, 0.8)',  # Semi-transparent white background
#         bordercolor='rgba(0, 0, 0, 0.2)',
#         borderwidth=1,
#         font=dict(size=9),  # Smaller font to save space
#         tracegroupgap=5,  # Reduce space between items (default is ~10-15)

#     ),
#     # Reduce margins significantly
#     margin=dict(
#         l=50,   # Left margin
#         r=20,   # Right margin (reduced since legend is inside plot area)
#         t=30,   # Top margin
#         b=50    # Bottom margin (for x-axis label)
#     ),
#     # Optional: make figure more compact
#     height=400,  # Reduce if needed
#     width=600
# )

fig.update_layout(
    showlegend=True,
    legend=dict(
        orientation="v",  # Make legend horizontal
        x=1,  # Center horizontally
        y=1,  # Position below plots (negative value)
        xanchor='right',
        yanchor='top',
        bgcolor='rgba(255, 255, 255, 0.8)',
        bordercolor='rgba(0, 0, 0, 0.2)',
        borderwidth=1,
        font=dict(size=9),
        tracegroupgap=5,
    ),
    # Increase bottom margin to make room for horizontal legend
    margin=dict(
        l=50,   
        r=20,   
        t=30,   
        b=50    # Increase bottom margin for legend
    ),
    height=250,
    width=600
)


# Update the x and y titles for the entire figure
fig.update_xaxes(title_text="Threshold", row=1, col=1)  # Left plot
fig.update_xaxes(title_text="Threshold", row=1, col=2)  # Right plot
fig.update_yaxes(title_text="FRRE Exceedance Rate", row=1, col=1)

fig.show()

In [ ]:
# TODO: Define path
path = ""
fig.write_image(path, format="pdf", width=600, height=250)

# HIT stats for Targeted Attack

In [ ]:
import pickle
import tempfile


def get_stats_for_attacker_id(attacker_run_id):
    all_hit_target = []
    all_increase_regret = []
    client = mlflow.MlflowClient()
    # Get the attacked models name 
    attacked_models_name = client.get_run(attacker_run_id).data.params["attacked_models_name"]
    print(f"Attacked Model: {attacked_models_name}")
    # Print the name of the attacker
    with tempfile.TemporaryDirectory() as tmpdir:
        path = mlflow.artifacts.download_artifacts(run_id = attacker_run_id, artifact_path="stats/stats.pkl", dst_path= tmpdir)
        with open(path, "rb") as f:
            stats = pickle.load(f)
        for nr, sample in enumerate(stats):
            all_hit_target.extend(sample["hit_target"])
            for i,regret in enumerate(sample["cur_regret"][1:]):
                if regret > sample["cur_regret"][i-1]:
                    all_increase_regret.append(True)
                else:
                    all_increase_regret.append(False)
    return np.mean(all_hit_target), np.mean(all_increase_regret)

def print_stats_for_all_iterative_attackers():
    data = load_data("Warcraft")
    iterative_attackers = data[data["params.attacker"] == "IterativeTargetedRegretMaximizationAttack"]
    for idx, row in iterative_attackers.iterrows():
        print(f"Attacker Sig: {row['attacker_sig']}")
        hit_target, increase_regret = get_stats_for_attacker_id(row["run_id"])
        print(f"Hit Target: {hit_target}, Increase Regret: {increase_regret}")
        print("-" * 20)

print_stats_for_all_iterative_attackers()